# Heuristic Goose - New non-NN baseline

- **Intention:** Try simple solutions first.
- **Notebook For:** Begineers to this Competition OR someone looking for baseline.
- **Approach:** Define a simple agent which performs following oprations each step, 
    1. Valid transitions: (a) do not collide with any agent, (b) do not make an action which is opposite to your own last action.
    2. Find the nearest food from my position
    3. Find the actions (from valid transitions) such that the distance from nearest food becomes smaller.
- **Where does this fails:** Majority times, due to greedy nature, it colides with an agent when both are at distance of 1 from the food.

### Import packages

In [ ]:
# import
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col, translate
from kaggle_environments import evaluate, make, utils
from joblib import Parallel, delayed
from typing import *
import numpy as np
import random

### Original baseline agent

- As defined in the Problem definition. 

In [ ]:
def baseline_agent(obs_dict, config_dict):
    """This agent always moves toward observation.food[0] but does not take advantage of board wrapping"""
    # convert 
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    # get stats
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)
    food = observation.food[0]
    food_row, food_column = row_col(food, configuration.columns)
    # choose action
    if food_row > player_row:
        return Action.SOUTH.name
    if food_row < player_row:
        return Action.NORTH.name
    if food_column > player_column:
        return Action.EAST.name
    return Action.WEST.name

## Our Custom agent -- Heuristic Goose

In [ ]:
%%writefile heuristic_goose.py

# import
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col, translate
from kaggle_environments import evaluate, make, utils
from typing import *
import numpy as np
import random

# my hitorical actions
my_actions = [None]

def find_valid_moves(geese_position, my_goose_index, my_last_action, config_col, config_rows):
    """Return list of valid moves. A valid move is one which 
    1. Doesn't colide with any agent's body 
    2. It isn't the opposite of my last action
    """
    # all possible actions
    actions = [Action.NORTH, Action.SOUTH, Action.WEST, Action.EAST]
    # find opposite of my last action -- invalid move
    my_last_action_opposite = my_last_action.opposite() if my_last_action is not None else None
    # get agent's position (head position)
    my_position = geese_position[my_goose_index][0]
    # find occupied positions
    occupied_positions = set([pos for one_goose_position in geese_position for pos in one_goose_position])
    # find translated position after taking actions
    possible_transition = [(action, translate(my_position, action, config_col, config_rows)) for action in actions if action is not my_last_action_opposite]
    # find valid transitions of (action, final_pos) tuples
    valid_transitions = [trans for trans in possible_transition if trans[1] not in occupied_positions]
    #
    return valid_transitions
    
def find_distance(distance_to, distance_from_list, columns):
    """Find the distance to a point from multiple other points.
    Based on Kaggle original code with same name in hungry_geese.py; 
    """
    row, column = row_col(distance_to, columns)
    return list(
        abs(row - pos_row) + abs(column - pos_col)
        for pos in distance_from_list
        for pos_row, pos_col in [row_col(pos, columns)]
    )

# custom agent
def agent(obs_dict, config_dict):
    """This is custom heuristic based agent
    Ver1: Please just don't colide :) -- Skipping dumb moves
    """
    global my_actions
    # convert 
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    # get stats
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)
    # find valid transitions
    valid_transitions = find_valid_moves(observation.geese, player_index, my_actions[-1], configuration.columns, configuration.rows)
    if len(valid_transitions) != 0:
        # find nearest food
        food_distance = find_distance(player_goose[0], observation.food, configuration.columns)
        nearest_food_pos = observation.food[np.argmin(food_distance)]
        # find the next valid action which is closest to the nearest food
        next_position_food_distance = find_distance(nearest_food_pos, [x[1] for x in valid_transitions], configuration.columns)
        next_action = valid_transitions[np.argmin(next_position_food_distance)][0]
    else:
        next_action = my_actions[-1] # we have already lost :(
    # update my_actions
    my_actions.append(next_action)
    #
    return next_action.name

## Play with agent

In [ ]:
env = make("hungry_geese", debug=True)
run_stats = env.run([
                baseline_agent,
                baseline_agent,
                "heuristic_goose.py", 
                "heuristic_goose.py",
            ])
env.render(mode="ipython", width=500, height=450)

### Print the run stats (for debugging)

In [ ]:
# run_stats[1:]

### Evaluate (get reward)

In [ ]:
evaluate("hungry_geese", [
    baseline_agent, 
    baseline_agent,
    "heuristic_goose.py", 
    "heuristic_goose.py", 
])

### Parallel evaluate

- Run evaluate 100 times to validate that the current approach is better than original baseline
- Ideally, the average reward points at the last 2 positions (heuristic) should be higher than first 2 (original baseline)
- Code inspired by: [Here](https://www.kaggle.com/jamesmcguigan/hungry-geese-go-west)

In [ ]:
# variable
trials = 100

# run parallel test for 100
results = Parallel()( 
    delayed(evaluate)("hungry_geese", [
        baseline_agent, 
        baseline_agent, 
        "heuristic_goose.py",
        "heuristic_goose.py", 
    ], num_episodes=1) 
for _ in range(trials) )


print("mean", np.mean(results, axis=0).astype(np.int).flatten())
print("max ", np.max( results, axis=0).astype(np.int).flatten())